In [2]:
import torch
import d2l as d2l
from torch import nn

In [3]:
def corr2d(X, k):
    """ 计算二维互相关运算 """
    h, w = k.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * k).sum()
    return Y

In [4]:
class Conv2D(nn.Module):
    """ 初始化卷积层 """
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## 学习卷积核

In [5]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)     # 构造一个二维卷积层，具有一个输出通道和形状为（1,2）的卷积核
X = torch.ones((6, 8))
X[:, 2:6] = 0
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat-Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')

conv2d.weight.data.reshape((1, 2))

batch 2, loss 6.151
batch 4, loss 1.392
batch 6, loss 0.381
batch 8, loss 0.124
batch 10, loss 0.046


tensor([[ 0.9676, -1.0099]])

## 填充和步幅

填充可以使得输入和输出的维度保持一致，防止像素的丢失，padding一般为（kernel_size-1）/2

步幅可以加快计算速度，缩短采样次数

In [6]:
def comp_conv2d(conv2d, X):
    X = X.reshape((1, 1) + X.shape)  # （1,1）表示批量大小和通道数都是1
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])  # 省略了前两个维度


conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [7]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

## 多输入通道和多输出通道

In [15]:
def corr2d_multi_in(X, K):
    """ 遍历X和K的第0个维度再把它们加在一起 """
    return sum(corr2d(x, k) for x, k in zip(X, K))


X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 图像池化

降低卷积层对位置的敏感性，同时降低对空间降采样表示的敏感性

In [16]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i:i + p_h, j:j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i:i + p_h, j:j + p_w].mean()
    return Y


X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

在针对多个输入通道的图像时，池化是对各个通道单独运算，不像卷积层一样对输入进行汇总。